## 1. Access Modifiers (Convention)

In [ ]:
# Python uses naming conventions, not true access control

class Example:
    def __init__(self):
        self.public = "Anyone can access"    # Public
        self._protected = "Internal use"     # Protected (convention)
        self.__private = "Name mangled"      # Private (name mangling)

obj = Example()

# Public - freely accessible
print(f"Public: {obj.public}")

# Protected - accessible but "don't touch" warning
print(f"Protected: {obj._protected}")

# Private - name mangled, not directly accessible
# print(obj.__private)  # AttributeError!
print(f"Private (mangled): {obj._Example__private}")

In [ ]:
# Name mangling explained
class BankAccount:
    def __init__(self, balance):
        self.__balance = balance  # Private
    
    def get_balance(self):
        return self.__balance
    
    def deposit(self, amount):
        if amount > 0:
            self.__balance += amount

account = BankAccount(1000)

# Proper access
print(f"Balance: ${account.get_balance()}")

# Attempting direct access
# account.__balance = 1000000  # Creates NEW attribute, doesn't modify original!

# View all attributes
print(f"\nAttributes: {[a for a in dir(account) if 'balance' in a.lower()]}")

## 2. Properties (@property)

In [ ]:
# Properties allow attribute-like access with method control

class Circle:
    def __init__(self, radius):
        self._radius = radius
    
    @property
    def radius(self):
        """Getter for radius"""
        return self._radius
    
    @radius.setter
    def radius(self, value):
        """Setter for radius"""
        if value <= 0:
            raise ValueError("Radius must be positive")
        self._radius = value
    
    @property
    def diameter(self):
        """Computed property (read-only)"""
        return self._radius * 2
    
    @property
    def area(self):
        """Computed property"""
        import math
        return math.pi * self._radius ** 2

# Use like attributes
circle = Circle(5)
print(f"Radius: {circle.radius}")
print(f"Diameter: {circle.diameter}")
print(f"Area: {circle.area:.2f}")

# Set using assignment
circle.radius = 10
print(f"\nNew radius: {circle.radius}")
print(f"New diameter: {circle.diameter}")

# Validation happens automatically
try:
    circle.radius = -5
except ValueError as e:
    print(f"\nError: {e}")

In [ ]:
# Read-only property (no setter)
class Rectangle:
    def __init__(self, width, height):
        self._width = width
        self._height = height
    
    @property
    def width(self):
        return self._width
    
    @property
    def height(self):
        return self._height
    
    @property
    def area(self):
        """Read-only computed property"""
        return self._width * self._height

rect = Rectangle(10, 5)
print(f"Area: {rect.area}")

# Can't modify
try:
    rect.area = 100  # No setter!
except AttributeError as e:
    print(f"Error: can't set attribute")

In [ ]:
# Property with deleter
class User:
    def __init__(self, name):
        self._name = name
        self._email = None
    
    @property
    def email(self):
        return self._email
    
    @email.setter
    def email(self, value):
        if value and '@' not in value:
            raise ValueError("Invalid email format")
        self._email = value
    
    @email.deleter
    def email(self):
        print("Deleting email...")
        self._email = None

user = User("Alice")
user.email = "alice@example.com"
print(f"Email: {user.email}")

del user.email
print(f"Email after delete: {user.email}")

## 3. Practical Encapsulation Examples

In [ ]:
# Temperature with automatic conversion
class Temperature:
    def __init__(self, celsius=0):
        self._celsius = celsius
    
    @property
    def celsius(self):
        return self._celsius
    
    @celsius.setter
    def celsius(self, value):
        if value < -273.15:
            raise ValueError("Temperature below absolute zero!")
        self._celsius = value
    
    @property
    def fahrenheit(self):
        return self._celsius * 9/5 + 32
    
    @fahrenheit.setter
    def fahrenheit(self, value):
        self.celsius = (value - 32) * 5/9  # Uses celsius setter validation
    
    @property
    def kelvin(self):
        return self._celsius + 273.15
    
    @kelvin.setter
    def kelvin(self, value):
        self.celsius = value - 273.15

# Use
temp = Temperature(25)
print(f"{temp.celsius}°C = {temp.fahrenheit}°F = {temp.kelvin}K")

temp.fahrenheit = 98.6
print(f"Body temp: {temp.celsius:.1f}°C")

temp.kelvin = 373.15
print(f"Boiling water: {temp.celsius:.1f}°C = {temp.fahrenheit:.1f}°F")

In [ ]:
# Person with age validation
class Person:
    def __init__(self, name, birth_year):
        self.name = name
        self._birth_year = birth_year
    
    @property
    def birth_year(self):
        return self._birth_year
    
    @birth_year.setter
    def birth_year(self, value):
        from datetime import datetime
        current_year = datetime.now().year
        
        if value > current_year:
            raise ValueError("Birth year cannot be in the future")
        if value < 1900:
            raise ValueError("Birth year seems too old")
        
        self._birth_year = value
    
    @property
    def age(self):
        """Computed age"""
        from datetime import datetime
        return datetime.now().year - self._birth_year
    
    @property
    def is_adult(self):
        """Check if adult"""
        return self.age >= 18

person = Person("Alice", 2000)
print(f"{person.name}:")
print(f"  Born: {person.birth_year}")
print(f"  Age: {person.age}")
print(f"  Adult: {person.is_adult}")

## 4. Complete Example: Bank Account with Full Encapsulation

In [ ]:
class BankAccount:
    """
    Secure bank account with proper encapsulation.
    
    - Balance is protected
    - All modifications go through methods
    - Transaction history is private
    """
    
    _next_account_number = 1000
    
    def __init__(self, owner, initial_balance=0):
        self._owner = owner
        self._balance = 0  # Use property setter for validation
        self.__transactions = []  # Private
        self._account_number = BankAccount._next_account_number
        BankAccount._next_account_number += 1
        
        # Initial deposit if provided
        if initial_balance > 0:
            self.deposit(initial_balance)
    
    # Properties
    @property
    def owner(self):
        return self._owner
    
    @property
    def account_number(self):
        return self._account_number
    
    @property
    def balance(self):
        return self._balance
    
    # No balance setter - must use deposit/withdraw
    
    @property
    def transaction_count(self):
        """Number of transactions (no access to details)"""
        return len(self.__transactions)
    
    # Methods
    def deposit(self, amount):
        """Deposit money"""
        if amount <= 0:
            raise ValueError("Deposit must be positive")
        
        self._balance += amount
        self._log_transaction("DEPOSIT", amount)
        return self._balance
    
    def withdraw(self, amount):
        """Withdraw money"""
        if amount <= 0:
            raise ValueError("Withdrawal must be positive")
        if amount > self._balance:
            raise ValueError("Insufficient funds")
        
        self._balance -= amount
        self._log_transaction("WITHDRAWAL", -amount)
        return self._balance
    
    def transfer(self, other_account, amount):
        """Transfer to another account"""
        if not isinstance(other_account, BankAccount):
            raise TypeError("Can only transfer to BankAccount")
        
        self.withdraw(amount)  # Will validate
        other_account.deposit(amount)
        print(f"Transferred ${amount:.2f} to account {other_account.account_number}")
    
    def get_statement(self):
        """Get account statement (controlled access to transactions)"""
        print(f"\n{'='*50}")
        print(f"  Account: {self._account_number}")
        print(f"  Owner: {self._owner}")
        print(f"{'='*50}")
        print(f"  {'Date':<20} {'Type':<12} {'Amount':>10}")
        print(f"  {'-'*44}")
        
        for t in self.__transactions[-10:]:  # Last 10
            sign = "+" if t['amount'] > 0 else ""
            print(f"  {t['date']:<20} {t['type']:<12} {sign}${abs(t['amount']):>8.2f}")
        
        print(f"  {'-'*44}")
        print(f"  {'Current Balance:':>32} ${self._balance:>8.2f}")
        print(f"{'='*50}\n")
    
    def _log_transaction(self, trans_type, amount):
        """Log transaction (protected method)"""
        from datetime import datetime
        self.__transactions.append({
            'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'type': trans_type,
            'amount': amount,
            'balance': self._balance
        })
    
    def __str__(self):
        return f"Account {self._account_number}: {self._owner} - ${self._balance:.2f}"

# Demo
print("🏦 BANK DEMO")
print("=" * 50)

# Create accounts
alice = BankAccount("Alice", 1000)
bob = BankAccount("Bob", 500)

print(f"Created: {alice}")
print(f"Created: {bob}")

# Operations
alice.deposit(250)
alice.withdraw(100)
alice.transfer(bob, 300)

# Statements
alice.get_statement()
bob.get_statement()

# Trying to cheat
print("\n🔒 Security tests:")

# Can't set balance directly
try:
    alice.balance = 1000000
except AttributeError:
    print("  ✅ Can't set balance directly")

# Can't access transactions
try:
    print(alice.__transactions)
except AttributeError:
    print("  ✅ Can't access private transactions")

## Summary

### Access Conventions:

| Prefix | Meaning | Access |
|--------|---------|--------|
| `name` | Public | Anyone |
| `_name` | Protected | Internal |
| `__name` | Private | Name mangled |

### Properties:

| Decorator | Purpose |
|-----------|----------|
| `@property` | Getter |
| `@name.setter` | Setter |
| `@name.deleter` | Deleter |

### Best Practices:
1. Use properties for controlled access
2. Validate in setters
3. Use `_` prefix for internal attributes
4. Use `__` for truly private data
5. Document your interface

### Next Lesson: Instance vs Class Variables/Methods